In [1]:
import IPython
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

Việc đề xuất cho khách hàng những sản phẩm thường được mua cùng nhau sẽ giúp mang lại hiệu quả bán chéo cao hơn, giúp doanh nghiệp gia tăng giá trị trung bình trên mỗi đơn hàng. 


Để đánh giá hiệu quả đề xuất các sản phẩm có thể bán chéo của doanh nghiệp thương mại điện tử từ bộ dữ liệu này đến khách hàng, nhóm sử dụng *thuật toán Apriori* để tìm ra các cặp sản phẩm (ở cấp độ danh mục) thường xuyên đi cùng nhau nhất trong những đơn hàng có từ 2 sản phẩm trở lên. Sau đó, nhóm sẽ khảo sát bảng dữ liệu event để đánh giá trong một session khách hàng có được dẫn đến những danh mục có khả năng bán chéo trong cùng một rổ sản phẩm hay không.

# Chuẩn bị dữ liệu

In [2]:
# Nhập các file chi tiết đơn hàng và item
df = pd.read_csv("order_details.csv")
df_item = pd.read_csv("items.csv")

In [3]:
# Tạo bảng đến số lượng sản phẩm trên một order
df_order = df.groupby('order_id')['id'].count().reset_index()
df_order = df_order[df_order['id']>1]
df_order

,order_id,id
0,1,2
2,3,3
3,4,3
5,6,2
6,7,3
...,...,...
124907,124908,2
124914,124915,4
124917,124918,2
124919,124920,2


Join bảng order với bảng số lượng trên từng order để thêm cột số lượng sản phẩm

In [4]:
df2 = df.merge(df_order,on='order_id',how='inner')

In [5]:
df2.head()

,id_x,order_id,item_id,inventory_id,sale_price,id_y
0,39441,27206,13606,106203,2.500000,3
1,39439,27206,2787,106198,40.000000,3
2,39440,27206,11006,106200,17.500000,3
3,33637,23239,13606,90590,2.500000,2
4,33636,23239,8495,90588,49.950001,2


In [7]:
# tạo bảng chi tiết đơn hàng với thông tin sản phẩm
df_detail = df2.merge(df_item, left_on='item_id', right_on='id', how='left')
df_detail.head()

,id_x,order_id,item_id,inventory_id,sale_price,id_y,id,cost,category,brand,retail_price,department,fulfillment_id
0,39441,27206,13606,106203,2.500000,3,13606,1.60750,Accessories,Scarf_tradinginc,2.500000,Women,3
1,39439,27206,2787,106198,40.000000,3,2787,17.50000,Active,adidas,40.000000,Women,4
2,39440,27206,11006,106200,17.500000,3,11006,9.06500,Intimates,Cosabella,17.500000,Women,10
3,33637,23239,13606,90590,2.500000,2,13606,1.60750,Accessories,Scarf_tradinginc,2.500000,Women,3
4,33636,23239,8495,90588,49.950001,2,8495,22.52695,Outerwear & Coats,2b by bebe,49.950001,Women,5


In [9]:
# Số các đơn hàng
df_detail['order_id'].nunique()

37374

In [10]:
# Kiểm tra các giá trị null
df_detail.isnull().sum()

id_x               0
order_id           0
item_id            0
inventory_id       0
sale_price         0
id_y               0
id                 0
cost               0
category           0
brand             72
retail_price       0
department         0
fulfillment_id     0
dtype: int64

# Thuật toán Apriori

In [11]:
# import thư viện chứa thuật toán apriori
from mlxtend.frequent_patterns import association_rules, apriori

Thuật toán Apriori giúp tìm ra các quy luật bên trong một tập dữ liệu gồm các sản phẩm có thể đi cùng nhau trong các đơn hàng. Trường dữ liệu được sử dụng trong quá trình sử dựng thuật toán này là Category (danh mục sản phẩm).

In [17]:
# Tạo bảng đơn hàng với các danh mục sản phẩm
transactions_str = df_detail.groupby(['order_id', 'category'])['item_id'].count().reset_index(name ='Count')
transactions_str

,order_id,category,Count
0,1,Pants,1
1,1,Sleep & Lounge,1
2,3,Shorts,1
3,3,Tops & Tees,1
4,3,Underwear,1
...,...,...,...
88178,124918,Sleep & Lounge,1
88179,124920,Jumpsuits & Rompers,1
88180,124920,Socks & Hosiery,1
88181,124923,Accessories,1


In [18]:
my_basket = transactions_str.pivot_table(index='order_id', columns='category', values='Count', aggfunc='sum').fillna(0)
my_basket.head()

category,Accessories,Active,Blazers & Jackets,Clothing Sets,Dresses,Fashion Hoodies & Sweatshirts,Intimates,Jeans,Jumpsuits & Rompers,Leggings,...,Skirts,Sleep & Lounge,Socks,Socks & Hosiery,Suits,Suits & Sport Coats,Sweaters,Swim,Tops & Tees,Underwear
order_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [19]:
def encode(x):
    if x<=0:
        return 0
    if x>=1:
        return 1

# applying the function to the dataset

my_basket_sets = my_basket.applymap(encode)
my_basket_sets.head()

category,Accessories,Active,Blazers & Jackets,Clothing Sets,Dresses,Fashion Hoodies & Sweatshirts,Intimates,Jeans,Jumpsuits & Rompers,Leggings,...,Skirts,Sleep & Lounge,Socks,Socks & Hosiery,Suits,Suits & Sport Coats,Sweaters,Swim,Tops & Tees,Underwear
order_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
6,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [23]:
frequent_items = apriori(my_basket_sets, min_support = 0.01,use_colnames = True)
frequent_items

c:\Users\ASUS\AppData\Local\Programs\Python\Python39\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.133703,(Accessories)
1,0.119334,(Active)
2,0.043586,(Blazers & Jackets)
3,0.071440,(Dresses)
4,0.154626,(Fashion Hoodies & Sweatshirts)
...,...,...
106,0.017552,"(Tops & Tees, Sweaters)"
107,0.012843,"(Underwear, Sweaters)"
108,0.017151,"(Tops & Tees, Swim)"
109,0.012174,"(Swim, Underwear)"


In [24]:
rules = association_rules(frequent_items, metric = "lift", min_threshold = 0.1)
rules.sort_values('confidence', ascending = False, inplace = True)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
41,(Blazers & Jackets),(Intimates),0.043586,0.159549,0.010649,0.244322,1.531323,0.003695,1.112180,0.362782
66,(Leggings),(Intimates),0.041740,0.159549,0.010007,0.239744,1.502629,0.003347,1.105483,0.349070
69,(Maternity),(Intimates),0.064109,0.159549,0.015278,0.238314,1.493668,0.005049,1.103408,0.353147
75,(Plus),(Intimates),0.055386,0.159549,0.013137,0.237198,1.486675,0.004301,1.101794,0.346552
43,(Dresses),(Intimates),0.071440,0.159549,0.016937,0.237079,1.485926,0.005539,1.101622,0.352179
...,...,...,...,...,...,...,...,...,...,...
40,(Intimates),(Blazers & Jackets),0.159549,0.043586,0.010649,0.066745,1.531323,0.003695,1.024815,0.412838
155,(Swim),(Socks),0.150238,0.082383,0.010007,0.066607,0.808503,-0.002370,0.983098,-0.217974
72,(Intimates),(Pants & Capris),0.159549,0.047011,0.010515,0.065906,1.401928,0.003015,1.020228,0.341122
67,(Intimates),(Leggings),0.159549,0.041740,0.010007,0.062720,1.502629,0.003347,1.022384,0.398000


In [25]:
rules.sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
41,(Blazers & Jackets),(Intimates),0.043586,0.159549,0.010649,0.244322,1.531323,0.003695,1.112180,0.362782
66,(Leggings),(Intimates),0.041740,0.159549,0.010007,0.239744,1.502629,0.003347,1.105483,0.349070
69,(Maternity),(Intimates),0.064109,0.159549,0.015278,0.238314,1.493668,0.005049,1.103408,0.353147
75,(Plus),(Intimates),0.055386,0.159549,0.013137,0.237198,1.486675,0.004301,1.101794,0.346552
43,(Dresses),(Intimates),0.071440,0.159549,0.016937,0.237079,1.485926,0.005539,1.101622,0.352179
...,...,...,...,...,...,...,...,...,...,...
40,(Intimates),(Blazers & Jackets),0.159549,0.043586,0.010649,0.066745,1.531323,0.003695,1.024815,0.412838
155,(Swim),(Socks),0.150238,0.082383,0.010007,0.066607,0.808503,-0.002370,0.983098,-0.217974
72,(Intimates),(Pants & Capris),0.159549,0.047011,0.010515,0.065906,1.401928,0.003015,1.020228,0.341122
67,(Intimates),(Leggings),0.159549,0.041740,0.010007,0.062720,1.502629,0.003347,1.022384,0.398000


Các rule có giá trị confidence cao nhất vào khoảng 24%. Để tìm ra các rule có khả năng làm tăng doanh số bán của một số category nhất định, nhóm chọn threshold cho chỉ số lift là 1.4, confidence tối thiểu là 20%.

In [28]:
rules[rules['lift']>1.4]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
41,(Blazers & Jackets),(Intimates),0.043586,0.159549,0.010649,0.244322,1.531323,0.003695,1.112180,0.362782
66,(Leggings),(Intimates),0.041740,0.159549,0.010007,0.239744,1.502629,0.003347,1.105483,0.349070
69,(Maternity),(Intimates),0.064109,0.159549,0.015278,0.238314,1.493668,0.005049,1.103408,0.353147
75,(Plus),(Intimates),0.055386,0.159549,0.013137,0.237198,1.486675,0.004301,1.101794,0.346552
43,(Dresses),(Intimates),0.071440,0.159549,0.016937,0.237079,1.485926,0.005539,1.101622,0.352179
81,(Socks & Hosiery),(Intimates),0.048510,0.159549,0.010917,0.225041,1.410481,0.003177,1.084510,0.305859
73,(Pants & Capris),(Intimates),0.047011,0.159549,0.010515,0.223677,1.401928,0.003015,1.082604,0.300839
132,(Pants),(Underwear),0.093033,0.099347,0.013753,0.147829,1.488000,0.004510,1.056892,0.361598
133,(Underwear),(Pants),0.099347,0.093033,0.013753,0.138433,1.488000,0.004510,1.052695,0.364133
124,(Socks),(Pants),0.082383,0.093033,0.010970,0.133160,1.431328,0.003306,1.046292,0.328403


Dựa vào kết quả trên có thể chọn ra được những rule có ý nghĩa nhất để đề xuất bán chéo là:

Blazers & Jacket -> Intimates 

Leggings -> Intimates

Maternity -> Intimates

Plus -> Intimates

Dresses -> Intimates

Socks & Hosiery -> Intimates

Pants & Capris -> Intimates

# Kiểm tra các session có hành động xem sản phẩm và danh mục sản phẩm

Các event dạng department cung cấp thông tin về danh mục mà khách hàng đang xem, dạng product cung cấp thông tin thông qua item_id trong URI dạng 'product/item_id'. Dựa trên các thông tin này nhóm tiến hành phân tích những danh mục được khách hàng xem qua trong cùng một session.

In [29]:
# import bảng events
df_event = pd.read_csv("events.csv")

In [30]:
# tạo biến category chỉ danh mục khách hàng xem thông qua các trang department hoặc product
df_event['category']=''

In [31]:
# Lấy giá trị category từ path department
for i in range(len(df_event)):
    if(df_event['event_type'][i]=='department'):
        df_event['category'][i]=df_event['path'][i].split('/')[4]

C:\Users\ASUS\AppData\Local\Temp\ipykernel_9044\918507670.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_event['category'][i]=df_event['path'][i].split('/')[4]


In [64]:
# tách giá trị item_id từ path của URI trang product
df_event['item'] = ''
for i in range(len(df_event)):
    if(df_event['event_type'][i]=='product'):
        df_event['item'][i]=int(df_event['path'][i].split('/')[2])

C:\Users\ASUS\AppData\Local\Temp\ipykernel_18472\3138104294.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_event['item'][i]=int(df_event['path'][i].split('/')[2])


In [33]:
# Tạo dataframe df_item để lấy thông tin về danh mục sản phẩm
df_item = pd.read_csv("items.csv")

In [67]:
# tạo bảng id và category từ bảng item để đối chiếu danh mục với path trong event
df_category = df_item[['id','category']]
# xóa khoảng cách trong chuỗi category, sau đó lowercase
df_category['category'] = df_category['category'].str.replace(' ','').str.lower()
df_category

C:\Users\ASUS\AppData\Local\Temp\ipykernel_18472\477117473.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_category['category'] = df_category['category'].str.replace(' ','').str.lower()


,id,category
0,27569,swim
1,27445,swim
2,27457,swim
3,27466,swim
4,27481,swim
...,...,...
29115,1993,fashionhoodies&sweatshirts
29116,2062,fashionhoodies&sweatshirts
29117,2308,fashionhoodies&sweatshirts
29118,1852,fashionhoodies&sweatshirts


In [69]:
# Tạo df_new từ df_event có bổ sung thêm giá trị category cho các event dạng xem product
df_new = df_event.merge(df_category, left_on='item', right_on='id', how='left')

In [71]:
df_new[df_new['event_type']=='product'].head()

,id_x,cus_id,sequence_number,session_id,created_at,ip,city,state,postal,browser,traffic_src,path,event_type,category_x,item,id_y,category_y
181973,1325,79.0,5,a116b7b9-7fec-48e6-a0ce-496e0b09e7ca,2019-02-27 07:29:06 UTC,70.140.27.106,Leganés,Comunidad de Madrid,28915,Safari,Email,/product/9069,product,,9069,9069.0,socks&hosiery
181974,2283,146.0,2,185119b7-fb36-4886-945b-5eda935a4ad4,2021-02-20 07:19:17 UTC,174.77.117.68,São Paulo,São Paulo,02675-031,IE,Email,/product/5967,product,,5967,5967.0,leggings
181975,2614,167.0,2,fca1b214-cdac-4c1b-8b44-480432e5265e,2021-05-01 08:24:26 UTC,114.127.241.130,Tangshan,Zhejiang,312300,Safari,Email,/product/761,product,,761,761.0,tops&tees
181976,7892,555.0,2,1c502cf3-2028-4289-8eb6-516e6e797eb0,2022-05-06 03:31:14 UTC,92.90.57.55,Parkersburg,West Virginia,26101,Chrome,Email,/product/26450,product,,26450,26450.0,underwear
181977,7984,564.0,5,2822af19-763f-489f-be13-a30b4fd3b9b1,2022-03-25 06:35:53 UTC,138.142.216.187,Mount Prospect,Illinois,60056,Chrome,Email,/product/17861,product,,17861,17861.0,fashionhoodies&sweatshirts


In [72]:
# Nhập các giá trị category từ category_y vào category_x
for i in range(len(df_new)):
    if(df_new['event_type'][i]=='product'):
        df_new['category_x'][i]=df_new['category_y'][i]

C:\Users\ASUS\AppData\Local\Temp\ipykernel_18472\1971466910.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['category_x'][i]=df_new['category_y'][i]


In [34]:
# tạo dataframe mới từ df_new với các cột id_x, cus_id, created_at, session_id, event_type, item, category_x
df_category_session = df_new[['id_x','cus_id','created_at','session_id','event_type','item','category_x']]
#df_category_session.to_csv('category_session.csv',index=False)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_9044\2497812708.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_category_session = pd.read_csv("category_session.csv")


In [35]:
# df_category_session tail order by session_id
df_category_session.sort_values('session_id', ascending=False).head()

,id_x,cus_id,created_at,session_id,event_type,item,category_x
1592291,1138307,87742.0,2020-08-31 13:44:36 UTC,fffffab3-3934-4d3b-a603-ca215398f0d9,cart,NaN,NaN
1849091,1138303,87742.0,2020-08-31 13:42:00 UTC,fffffab3-3934-4d3b-a603-ca215398f0d9,product,17900.0,fashionhoodies&sweatshirts
1581333,1138304,87742.0,2020-08-31 13:42:23 UTC,fffffab3-3934-4d3b-a603-ca215398f0d9,cart,NaN,NaN
1575962,1138310,87742.0,2020-08-31 13:49:49 UTC,fffffab3-3934-4d3b-a603-ca215398f0d9,cart,NaN,NaN
2281626,1138311,87742.0,2020-08-31 13:51:23 UTC,fffffab3-3934-4d3b-a603-ca215398f0d9,department,NaN,fashionhoodies&sweatshirts


Để đánh giá việc người dùng có được xem các sản phẩm cùng một rổ sản phẩm (những danh mục bán chéo tiềm năng) không, nhóm lọc ra những dòng event thuộc loại department và product.

In [36]:
# drop các dòng có event_type không phải product hoặc department
df_category_session = df_category_session[df_category_session['category_x']!='']
df_category_session.sort_values('session_id', ascending=False).head()

,id_x,cus_id,created_at,session_id,event_type,item,category_x
1592291,1138307,87742.0,2020-08-31 13:44:36 UTC,fffffab3-3934-4d3b-a603-ca215398f0d9,cart,NaN,NaN
1849091,1138303,87742.0,2020-08-31 13:42:00 UTC,fffffab3-3934-4d3b-a603-ca215398f0d9,product,17900.0,fashionhoodies&sweatshirts
1581333,1138304,87742.0,2020-08-31 13:42:23 UTC,fffffab3-3934-4d3b-a603-ca215398f0d9,cart,NaN,NaN
1575962,1138310,87742.0,2020-08-31 13:49:49 UTC,fffffab3-3934-4d3b-a603-ca215398f0d9,cart,NaN,NaN
2281626,1138311,87742.0,2020-08-31 13:51:23 UTC,fffffab3-3934-4d3b-a603-ca215398f0d9,department,NaN,fashionhoodies&sweatshirts


In [37]:
# group by session_id với các cột session_id và category là mảng các category_x
df_category_session = df_category_session.groupby('session_id')['category_x'].apply(list).reset_index(name='category')
df_category_session

,session_id,category
0,00000447-903c-4b86-85c7-55edefb3ad73,"[outerwear&coats, outerwear&coats]"
1,000005b7-07e4-48d3-aecc-1556aa1de28e,"[pants&capris, nan, nan]"
2,000005f7-1446-44ce-abd2-4fc4955a9c3e,"[nan, nan, outerwear&coats]"
3,000006d5-2115-4b8e-9910-e98231623d54,[socks]
4,00000f42-fa0b-4fe9-8c4e-e87703fb5ac8,"[nan, nan, nan, intimates, intimates]"
...,...,...
680947,ffffe759-7ade-4334-9ebf-caf1de5ef26b,[jeans]
680948,fffff50c-9815-4abd-bd05-f9cdd379592b,"[nan, socks, socks]"
680949,fffff6bf-388c-4a67-a010-663c999cea85,[plus]
680950,fffff8a2-3063-4a6e-9c7e-bb8d6936e6ea,"[nan, nan, tops&tees, tops&tees, nan, tops&tee..."


In [42]:
# Sử dụng vòng lặp đánh dấu các session có các danh mục bán chéo tiềm năng
df_category_session['cross_sell_potential'] = 0
for i in range(len(df_category_session)):
    if('intimates' in df_category_session['category'][i]):
        for j in range(len(df_category_session['category'][i])):
            if(df_category_session['category'][i][j] in ['blazers&jacket','leggings','maternity','plus','dresses','socks&hosiery','pants&capris']):
                df_category_session['cross_sell_potential'][i]=1

In [44]:
# Đếm các giá trị cross_sell_potential
df_category_session['cross_sell_potential'].value_counts()

0    680952
Name: cross_sell_potential, dtype: int64

Có thể thấy rằng không có session nào chứa các danh mục thuộc cùng một rổ danh mục tiềm năng như đã phát hiện bằng thuật toán Apriori. Điều này làm giảm tiềm năng bán chéo khi mà khách hàng không thấy được những danh mục họ có khả năng sẽ mua kèm khi đã xem xét có ý định mua những sản phẩm thuộc một danh mục khác của rổ.